In [1]:
!pip install pennylane-lightning-gpu
!pip install pennylane pennylane-lightning-gpu custatevec-cu12 --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.2
    Uninstalling autograd-1.6.2:
      Successfully uninstalled autograd-1.6.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 36.3 MB/s eta 0:00:00


In [2]:
!pip install jax
!pip install scikit-learn
!pip install pennylane

In [6]:
import sklearn.datasets
import numpy as np
import torch
from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane.operation import Tensor

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from collections import defaultdict

results = defaultdict(list)
from jax.config import config
config.update("jax_enable_x64", True)

import jax
import jax.numpy as jnp

results["quantum"] = defaultdict(list)
results["classical"] = defaultdict(list)

for no_of_classes in [2,3,4,5,6]:
    for p in [2,3,4,5]:
        X = sklearn.datasets.load_digits(n_class=no_of_classes)

        scaler = StandardScaler().fit(X.data)
        X_scaled = scaler.transform(X.data)

        pca = PCA(n_components=p)
        pca.fit(X_scaled)
        X_transform_pca = pca.transform(X_scaled)

        y_scaled = 2 * (X.target - 0.5)

        X_train, X_test, y_train, y_test = train_test_split(X_transform_pca, y_scaled)

        for x in range(len(X_train)):
            X_train[x] = X_train[x]/np.linalg.norm(X_train[x])

        for x in range(len(X_test)):
            X_test[x] = X_test[x]/np.linalg.norm(X_test[x])

        n_qubits = len(X_train[0])

        print('Number of qubits being used : ', n_qubits)

        dev_kernel = qml.device("default.qubit.jax", wires=n_qubits)

        projector = np.zeros((2**n_qubits, 2**n_qubits))
        projector[0, 0] = 1

        @jax.jit
        @qml.qnode(dev_kernel, interface="jax")
        def kernel(x):
            AngleEmbedding(x[0], wires=range(n_qubits))
            qml.adjoint(AngleEmbedding)(x[1], wires=range(n_qubits))
            return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

        jit_circuit = jax.jit(kernel)

        def kernel_matrix(A, B):
            return np.array([[jit_circuit(jnp.array([a, b])) for b in B] for a in A])

        qclassifier = SVC(kernel=kernel_matrix).fit(X_train, y_train)

        predictions = qclassifier.predict(X_test)
        score = accuracy_score(predictions, y_test)

        results['quantum'][no_of_classes].append((p,score))
        print ("Q:", no_of_classes, p, score)

        clf = SVC(kernel='rbf')
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        score = accuracy_score(predictions, y_test)
        results['classical'][no_of_classes].append((p, score))
        print ("C", no_of_classes, p, score)

print (results)

Number of qubits being used :  2
Q: 2 2 1.0
C 2 2 1.0
Number of qubits being used :  3
Q: 2 3 0.9888888888888889
C 2 3 0.9888888888888889
Number of qubits being used :  4
Q: 2 4 1.0
C 2 4 1.0
Number of qubits being used :  5
Q: 2 5 1.0
C 2 5 1.0
Number of qubits being used :  2
Q: 3 2 0.9333333333333333
C 3 2 0.9407407407407408
Number of qubits being used :  3
Q: 3 3 0.9407407407407408
C 3 3 0.9481481481481482
Number of qubits being used :  4
Q: 3 4 0.9481481481481482
C 3 4 0.9407407407407408
Number of qubits being used :  5
Q: 3 5 0.9851851851851852
C 3 5 0.9777777777777777
Number of qubits being used :  2
Q: 4 2 0.7555555555555555
C 4 2 0.7555555555555555
Number of qubits being used :  3
Q: 4 3 0.8833333333333333
C 4 3 0.8833333333333333
Number of qubits being used :  4
Q: 4 4 0.9222222222222223
C 4 4 0.9222222222222223
Number of qubits being used :  5
Q: 4 5 0.9666666666666667
C 4 5 0.9666666666666667
Number of qubits being used :  2
Q: 5 2 0.8407079646017699
C 5 2 0.845132743362831

In [14]:
for i in results['quantum'].keys():
  print('\nNo of classes (in dataset): ', i)
  print ('PCA dim\t Quantum\t Classical')
  for j in range(len(results['quantum'][i])):
    pca, accuracy = results['quantum'][i][j]
    print(pca,'\t', "%.5f" %np.round(accuracy,5), '\t',"%.5f" %np.round(results['classical'][i][j][1],5))


No of classes (in dataset):  2
PCA dim	 Quantum	 Classical
2 	 1.00000 	 1.00000
3 	 0.98889 	 0.98889
4 	 1.00000 	 1.00000
5 	 1.00000 	 1.00000

No of classes (in dataset):  3
PCA dim	 Quantum	 Classical
2 	 0.93333 	 0.94074
3 	 0.94074 	 0.94815
4 	 0.94815 	 0.94074
5 	 0.98519 	 0.97778

No of classes (in dataset):  4
PCA dim	 Quantum	 Classical
2 	 0.75556 	 0.75556
3 	 0.88333 	 0.88333
4 	 0.92222 	 0.92222
5 	 0.96667 	 0.96667

No of classes (in dataset):  5
PCA dim	 Quantum	 Classical
2 	 0.84071 	 0.84513
3 	 0.94248 	 0.94248
4 	 0.96018 	 0.96018
5 	 0.94690 	 0.94690

No of classes (in dataset):  6
PCA dim	 Quantum	 Classical
2 	 0.73063 	 0.73063
3 	 0.87823 	 0.88192
4 	 0.95941 	 0.95572
5 	 0.95941 	 0.95941
